📊 Principais Insights Possíveis
1. Análise de Churn (Evasão de Clientes)
Campos: churn, satisfacao_geral, atrasos_pagamento, usa_app_mobile, participa_programa_fidelidade
Insights:

Taxa de churn por nível de satisfação
Correlação entre atrasos e evasão
Impacto do uso do app mobile na retenção
Efetividade do programa de fidelidade

2. Segmentação de Clientes
Campos: idade, renda_mensal, saldo_conta, score_credito, tipo_conta
Insights:

Perfis de clientes por faixa etária
Segmentos por capacidade financeira
Clusters de comportamento (alto valor, risco, potencial)
Preferências de produtos por perfil demográfico

3. Comportamento Transacional
Campos: numero_transacoes_mes, media_valor_transacao, tipo_conta, usa_app_mobile
Insights:

Padrões de uso por tipo de conta
Valor médio transacional por segmento
Impacto do app mobile no volume de transações
Identificação de clientes inativos ou super-ativos

4. Análise de Risco e Crédito
Campos: score_credito, atrasos_pagamento, possui_cartao_credito, renda_mensal, saldo_conta
Insights:

Perfil de inadimplência
Clientes elegíveis para upgrade de produtos
Correlação entre score e comportamento financeiro
Oportunidades de cross-selling (cartão de crédito)

5. Satisfação do Cliente
Campos: satisfacao_geral, tipo_conta, usa_app_mobile, participa_programa_fidelidade
Insights:

Drivers de satisfação
NPS estimado por segmento
Pontos de melhoria por produto
Relação entre engajamento digital e satisfação

6. Engajamento Digital
Campos: usa_app_mobile, numero_transacoes_mes, idade, tipo_conta
Insights:

Taxa de adoção digital por geração
Correlação entre uso do app e atividade
Oportunidades de migração para canais digitais


# Etapa 1) Extração dos Dados

## Fonte de Dados
- Arquivo: `clientes_bancarios.csv`
- Encoding: UTF-8
- Tipo: CSV (Comma Separated Values)

In [ ]:
import pandas as pd

# Lê o arquivo CSV e carrega em um DataFrame
df_clientes = pd.read_csv('clientes_bancarios.csv', encoding='utf-8')

print("DataFrame carregado com sucesso!")
print("\nPrimeiras 5 linhas do DataFrame:")
print(df_clientes.head())

# Etapa 2) Preparação dos dados

## Tratamento Completo de Tipos
Conversão e validação de todos os campos:
- Inteiros: idade, score_credito, numero_transacoes_mes
- Decimal(16,2): renda_mensal, saldo_conta
- Strings (categóricas): genero, possui_cartao_credito, tipo_conta, usa_app_mobile, participa_programa_fidelidade, churn

In [ ]:
def converter_tipos_dados(df):
    """
    Função para converter os tipos de dados do DataFrame
    """
    df_convertido = df.copy()
    
    # Definir tipos de colunas
    colunas_int = ['idade', 'score_credito', 'numero_transacoes_mes']
    colunas_decimal = ['renda_mensal', 'saldo_conta']
    colunas_texto = ['possui_cartao_credito', 'usa_app_mobile', 
                     'participa_programa_fidelidade', 'churn']
    
    # Converter colunas inteiras
    for col in colunas_int:
        if col in df_convertido.columns:
            df_convertido[col] = pd.to_numeric(df_convertido[col], errors='coerce').fillna(0).astype('int64')
    
    # Converter colunas decimais
    for col in colunas_decimal:
        if col in df_convertido.columns:
            df_convertido[col] = pd.to_numeric(df_convertido[col], errors='coerce').fillna(0).astype('float64').round(2)
    
    # Converter colunas de texto
    for col in colunas_texto:
        if col in df_convertido.columns:
            df_convertido[col] = df_convertido[col].astype('string')
    
    return df_convertido


# Testar as funções com os dados
print("=== Conversão de Tipos ===")
df_convertido = converter_tipos_dados(df_clientes)
print("\nTipos de dados antes da conversão:")
print(df_clientes.dtypes)
print("\nTipos de dados após conversão:")
print(df_convertido.dtypes)



## Tratamento de Valores de Domínio

Validação de campos categóricos:
- possui_cartao_credito → [Sim, Não]
- usa_app_mobile → [Sim, Não]
- participa_programa_fidelidade → [Sim, Não]
- churn → [Sim, Não]

In [ ]:

def validar_valores(df):
    """
    Função para validar e padronizar os valores do DataFrame
    """
    df_validado = df.copy()
    
    # Definir valores válidos para campos categóricos
    colunas_categoricas = {
        'possui_cartao_credito': ['Sim', 'Não'],
        'usa_app_mobile': ['Sim', 'Não'],
        'participa_programa_fidelidade': ['Sim', 'Não'],
        'churn': ['Sim', 'Não']
    }
    
    # Validar e padronizar colunas categóricas
    for col, valores_validos in colunas_categoricas.items():
        if col in df_validado.columns:
            # Limpar e padronizar strings
            df_validado[col] = df_validado[col].str.lower().str.strip()
            
            # Mapear variações comuns para valores padrão
            mapa_valores = {
                'yes': 'sim', 'y': 'sim', 's': 'sim', '1': 'sim', 'true': 'sim',
                'no': 'não', 'n': 'não', '0': 'não', 'false': 'não'
            }
            df_validado[col] = df_validado[col].map(mapa_valores).fillna(df_validado[col])
            
            # Substituir valores inválidos por valor padrão
            mascara = ~df_validado[col].str.lower().isin([v.lower() for v in valores_validos])
            df_validado.loc[mascara, col] = valores_validos[0]
    
    return df_validado
print("\n=== Validação de Valores ===")
df_tratado = validar_valores(df_convertido)
print("\nAmostra dos dados após validação:")
print(df_tratado.head())


# Etapa 3) Aplicação de Regras

## Segmentação de Clientes
Campos: idade, renda_mensal, saldo_conta, score_credito, tipo_conta
Insights:

Perfis de clientes por faixa etária
Segmentos por capacidade financeira
Clusters de comportamento (alto valor, risco, potencial)
Preferências de produtos por perfil demográfico

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

def analisar_segmentos_clientes(df):
    """
    Função para análise detalhada de segmentação de clientes
    """
    resultados = {}
    
    # 1. Análise por Faixa Etária
    df['faixa_etaria'] = pd.cut(df['idade'], 
                               bins=[0, 25, 35, 50, 65, 100],
                               labels=['18-25', '26-35', '36-50', '51-65', '65+'])
    
    perfil_idade = df.groupby('faixa_etaria').agg({
        'renda_mensal': 'mean',
        'saldo_conta': 'mean',
        'score_credito': 'mean'
    }).round(2)
    
    # 2. Segmentação por Capacidade Financeira
    df['segmento_financeiro'] = pd.qcut(df['renda_mensal'], 
                                      q=3, 
                                      labels=['Básico', 'Intermediário', 'Premium'])
    
    perfil_financeiro = df.groupby('segmento_financeiro').agg({
        'saldo_conta': 'mean',
        'score_credito': 'mean',
        'idade': 'mean'
    }).round(2)
    
    # 3. Clusters de Comportamento
    # Preparar dados para clustering
    features_cluster = ['idade', 'renda_mensal', 'saldo_conta', 'score_credito']
    X = df[features_cluster].copy()
    
    # Normalizar dados
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Aplicar K-means
    kmeans = KMeans(n_clusters=3, random_state=42)
    df['cluster'] = kmeans.fit_predict(X_scaled)
    
    # Análise dos clusters
    perfil_clusters = df.groupby('cluster').agg({
        'idade': 'mean',
        'renda_mensal': 'mean',
        'saldo_conta': 'mean',
        'score_credito': 'mean'
    }).round(2)
    
    # 4. Análise por Tipo de Conta
    perfil_conta = df.groupby('tipo_conta').agg({
        'idade': 'mean',
        'renda_mensal': 'mean',
        'saldo_conta': 'mean',
        'score_credito': 'mean'
    }).round(2)
    
    # Armazenar resultados
    resultados['perfil_idade'] = perfil_idade
    resultados['perfil_financeiro'] = perfil_financeiro
    resultados['perfil_clusters'] = perfil_clusters
    resultados['perfil_conta'] = perfil_conta
    
    return resultados, df

# Realizar análise
resultados_segmentacao, df_segmentado = analisar_segmentos_clientes(df_tratado)

## Engajamento Digital

Objetivos
- Medir a taxa de adoção do app por geração (faixa etária)
- Avaliar a correlação entre uso do app e atividade (número de transações/mês)
- Identificar oportunidades de migração para canais digitais (clientes que não usam o app, mas são ativos)

Campos usados: `usa_app_mobile`, `numero_transacoes_mes`, `idade`, `tipo_conta`

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats


def analisar_engajamento_digital(df):
    """
    Analisa engajamento digital a partir de df (espera-se `df_tratado`).
    Retorna um dicionário com:
      - adoption_by_generation: taxa de adoção (%) por faixa etária
      - transacoes_por_app: estatísticas de transações por uso do app
      - adoption_by_tipo_conta: taxa de adoção por tipo de conta
      - correlacao_app_transacoes: coeficiente (point-biserial) entre uso do app e número de transações
      - oportunidades_migracao: DataFrame com clientes (amostra) que não usam o app mas têm alto volume de transações
    """
    resultados = {}
    df_copy = df.copy()

    # Verificar colunas mínimas
    required = ['usa_app_mobile', 'numero_transacoes_mes', 'idade', 'tipo_conta']
    missing = [c for c in required if c not in df_copy.columns]
    if missing:
        raise ValueError(f"Colunas faltando no DataFrame: {missing}")

    # Padronizar valores de usa_app_mobile (tolerante a maiúsculas/minúsculas)
    df_copy['usa_app_mobile_norm'] = df_copy['usa_app_mobile'].astype(str).str.lower().str.strip()
    df_copy['usa_app_mobile_flag'] = df_copy['usa_app_mobile_norm'].map(lambda x: 1 if x in ['sim','s','yes','1','true'] else 0)

    # 1) Taxa de adoção digital por geração (faixa etária)
    bins = [0, 25, 35, 50, 65, 120]
    labels = ['18-25', '26-35', '36-50', '51-65', '65+']
    # proteger idades fora do range
    df_copy['faixa_etaria_eng'] = pd.cut(df_copy['idade'].clip(lower=0), bins=bins, labels=labels, include_lowest=True)

    adoption_by_generation = (df_copy.groupby('faixa_etaria_eng')['usa_app_mobile_flag']
                              .mean() * 100).round(2).fillna(0)

    # 2) Correlação entre uso do app e atividade (numero_transacoes_mes)
    transacoes_por_app = (df_copy.groupby('usa_app_mobile_norm')['numero_transacoes_mes']
                          .agg(['mean','median','std','count']).round(2))

    # point-biserial correlation (usa_app_mobile_flag vs numero_transacoes_mes)
    try:
        corr_res = stats.pointbiserialr(df_copy['usa_app_mobile_flag'], df_copy['numero_transacoes_mes'])
        correlacao_app_transacoes = {
            'r': round(corr_res.correlation, 4) if hasattr(corr_res, 'correlation') else round(corr_res[0], 4),
            'pvalue': round(corr_res.pvalue, 4) if hasattr(corr_res, 'pvalue') else round(corr_res[1], 4)
        }
    except Exception:
        # fallback: Pearson on binary flag
        r, p = stats.pearsonr(df_copy['usa_app_mobile_flag'], df_copy['numero_transacoes_mes'])
        correlacao_app_transacoes = {'r': round(r, 4), 'pvalue': round(p, 4)}

    # 3) Taxa de adoção por tipo de conta
    adoption_by_tipo_conta = (df_copy.groupby('tipo_conta')['usa_app_mobile_flag'].mean() * 100).round(2)

    # 4) Oportunidades de migração: clientes que não usam o app (flag==0) mas têm alto número de transações
    threshold = df_copy['numero_transacoes_mes'].quantile(0.75)
    oportunidades = df_copy[(df_copy['usa_app_mobile_flag'] == 0) & (df_copy['numero_transacoes_mes'] >= threshold)].copy()

    # Agrupar oportunidades por tipo de conta para priorização
    oportunidades_agrupadas = (oportunidades.groupby('tipo_conta')
                               .agg({'numero_transacoes_mes': ['count','mean']}).round(2))

    # Amostra — mostrar até 20 clientes (se houver identificador, manter; senão, mostrar colunas relevantes)
    sample_cols = [c for c in ['idade','tipo_conta','numero_transacoes_mes','usa_app_mobile'] if c in oportunidades.columns]
    oportunidades_amostra = oportunidades[sample_cols].head(20)

    # Armazenar resultados
    resultados['adoption_by_generation'] = adoption_by_generation
    resultados['transacoes_por_app'] = transacoes_por_app
    resultados['correlacao_app_transacoes'] = correlacao_app_transacoes
    resultados['adoption_by_tipo_conta'] = adoption_by_tipo_conta
    resultados['oportunidades_agrupadas'] = oportunidades_agrupadas
    resultados['oportunidades_amostra'] = oportunidades_amostra

    return resultados, df_copy


# Executar a análise (gera resultados que podem ser usados pela próxima célula de visualização)
resultados_engajamento, df_engajamento = analisar_engajamento_digital(df_tratado)

# Etapa 4) Carga de Dados no Destino


In [28]:
# ========================================
# EXPORTAR DATAFRAME FINAL PARA CSV
# ========================================

# Nome do arquivo de saída
arquivo_saida_1 = f'segmento_cliente.csv'
arquivo_saida_2 = f'mkt_super_app.csv'


# Exportar o DataFrame segmentado (que contém todas as análises)
df_segmentado.to_csv(arquivo_saida_1, index=False, encoding='utf-8')
df_engajamento.to_csv(arquivo_saida_2, index=False, encoding='utf-8')

print(f"Arquivos CSVs {arquivo_saida_1} e {arquivo_saida_2} criados com sucesso!")

Arquivos CSVs segmento_cliente.csv e mkt_super_app.csv criados com sucesso!


# Etapa 5) Consumo - Analise dos Dados

## Segmentação de Clientes
Perfis de clientes por faixa etária
Segmentos por capacidade financeira
Clusters de comportamento (alto valor, risco, potencial)
Preferências de produtos por perfil demográfico

In [ ]:
# Criar figura para os gráficos com estilo padrão
plt.rcParams['figure.figsize'] = [15, 12]
fig, axes = plt.subplots(2, 2)

# 1. Distribuição por Faixa Etária
sns.barplot(x=df_segmentado['faixa_etaria'].value_counts().index, 
            y=df_segmentado['faixa_etaria'].value_counts().values,
            ax=axes[0,0])
axes[0,0].set_title('Distribuição por Faixa Etária')
axes[0,0].set_xlabel('Faixa Etária')
axes[0,0].set_ylabel('Quantidade de Clientes')
axes[0,0].tick_params(axis='x', rotation=45)

# 2. Renda Média por Segmento Financeiro
sns.barplot(x=df_segmentado.groupby('segmento_financeiro')['renda_mensal'].mean().index,
            y=df_segmentado.groupby('segmento_financeiro')['renda_mensal'].mean().values,
            ax=axes[0,1])
axes[0,1].set_title('Renda Média por Segmento')
axes[0,1].set_xlabel('Segmento')
axes[0,1].set_ylabel('Renda Média')

# 3. Visualização dos Clusters
scatter = axes[1,0].scatter(df_segmentado['renda_mensal'], 
                          df_segmentado['score_credito'],
                          c=df_segmentado['cluster'],
                          cmap='viridis')
axes[1,0].set_title('Clusters de Clientes')
axes[1,0].set_xlabel('Renda Mensal')
axes[1,0].set_ylabel('Score de Crédito')
axes[1,0].legend(*scatter.legend_elements(), title="Clusters")

# 4. Média de Saldo por Tipo de Conta
sns.barplot(x=df_segmentado.groupby('tipo_conta')['saldo_conta'].mean().index,
            y=df_segmentado.groupby('tipo_conta')['saldo_conta'].mean().values,
            ax=axes[1,1])
axes[1,1].set_title('Saldo Médio por Tipo de Conta')
axes[1,1].set_xlabel('Tipo de Conta')
axes[1,1].set_ylabel('Saldo Médio')
axes[1,1].tick_params(axis='x', rotation=45)

# Ajustar layout
plt.tight_layout()
plt.show()

# Imprimir análises detalhadas
print("\n=== Análise de Segmentação de Clientes ===")

print("\n1. Perfil por Faixa Etária:")
print(resultados_segmentacao['perfil_idade'])

print("\n2. Perfil por Segmento Financeiro:")
print(resultados_segmentacao['perfil_financeiro'])

print("\n3. Perfil dos Clusters:")
print(resultados_segmentacao['perfil_clusters'])

print("\n4. Perfil por Tipo de Conta:")
print(resultados_segmentacao['perfil_conta'])

## Engajamento Digital

Objetivos
- Medir a taxa de adoção do app por geração (faixa etária)
- Avaliar a correlação entre uso do app e atividade (número de transações/mês)
- Identificar oportunidades de migração para canais digitais (clientes que não usam o app, mas são ativos)


In [ ]:
# Visualizações e relatório de Engajamento Digital
plt.rcParams['figure.figsize'] = [14, 10]
fig, axes = plt.subplots(2, 2)

# 1) Adoção por geração
adopt_gen = resultados_engajamento['adoption_by_generation']
sns.barplot(x=adopt_gen.index, y=adopt_gen.values, ax=axes[0,0], palette='Blues_d')
axes[0,0].set_title('Taxa de Adoção do App por Faixa Etária (%)')
axes[0,0].set_xlabel('Faixa Etária')
axes[0,0].set_ylabel('Adoção (%)')
axes[0,0].tick_params(axis='x', rotation=45)

# 2) Estatísticas de transações por uso do app
trans_app = resultados_engajamento['transacoes_por_app']
trans_app.reset_index(inplace=True)
sns.barplot(x='usa_app_mobile_norm', y='mean', data=trans_app, ax=axes[0,1], palette='Greens_d')
axes[0,1].set_title('Média de Transações por Mês (por uso do app)')
axes[0,1].set_xlabel('Usa App (normalizado)')
axes[0,1].set_ylabel('Média transações/mês')
axes[0,1].tick_params(axis='x', rotation=45)

# 3) Adoção por tipo de conta
adopt_tipo = resultados_engajamento['adoption_by_tipo_conta']
adopt_tipo = adopt_tipo.sort_values(ascending=False)
sns.barplot(x=adopt_tipo.index, y=adopt_tipo.values, ax=axes[1,0], palette='Oranges_d')
axes[1,0].set_title('Taxa de Adoção do App por Tipo de Conta (%)')
axes[1,0].set_xlabel('Tipo de Conta')
axes[1,0].set_ylabel('Adoção (%)')
axes[1,0].tick_params(axis='x', rotation=45)

# 4) Oportunidades de migração por tipo de conta
opp = resultados_engajamento['oportunidades_agrupadas']
if not opp.empty:
    opp.columns = ['_'.join(col).strip() for col in opp.columns]
    opp = opp.reset_index()
    sns.barplot(x='tipo_conta', y='numero_transacoes_mes_count', data=opp, ax=axes[1,1], palette='Reds_d')
    axes[1,1].set_title('Clientes Não-App com Alto Volume (contagem por tipo de conta)')
    axes[1,1].set_xlabel('Tipo de Conta')
    axes[1,1].set_ylabel('Quantidade de Clientes')
    axes[1,1].tick_params(axis='x', rotation=45)
else:
    axes[1,1].text(0.5, 0.5, 'Sem oportunidades identificadas', ha='center')
    axes[1,1].set_axis_off()

plt.tight_layout()
plt.show()

# Relatório resumido em texto
print('\n=== Relatório: Engajamento Digital ===')
print('\n1) Adoção por geração (%):')
print(resultados_engajamento['adoption_by_generation'])

print('\n2) Estatísticas de transações por uso do app:')
print(resultados_engajamento['transacoes_por_app'])

print('\n3) Correlação (uso app vs transações):')
print(resultados_engajamento['correlacao_app_transacoes'])

print('\n4) Adoção por tipo de conta (%):')
print(resultados_engajamento['adoption_by_tipo_conta'])

print('\n5) Oportunidades de migração (resumo por tipo de conta):')
print(resultados_engajamento['oportunidades_agrupadas'])

print('\n6) Amostra de clientes candidatos à migração (até 20 linhas):')
print(resultados_engajamento['oportunidades_amostra'])